<img src="https://s3-ap-southeast-1.amazonaws.com/he-public-data/wordmark_black65ee464.png" width="700">

# Week 2 : Final Challenge

**Welcome to the final challenge!**  

In the previous notebook we've seen how we can use the VQC class in Aqua to classify the digits `0` and `1`. However, classifying `0` and `1` is relatively simple as digits `0` and `1` are easily distinguishable. `4` and `9` however, are notoriously similar, with a _loop_ on the top and a _line_ on the bottom. This can be corroborated looking at our 2-D t-SNE plot from the previous notebook (Fig.2), we see that `0` and `1` are clustered relatively far from each other making them easily distinguishable, however `4` and `9` are overlapping. In this challenge we are providing you with a dataset with digits reduced to **dimension 3**. For example, in Fig.1 we can see the dimension reduction of the 784 dimension vector for digit `4` into a dimension 3 feature vector. 

**Fig.1 : Features of the digit `4` after reducing dimension to 3:** 
<img src="https://s3-ap-southeast-1.amazonaws.com/he-public-data/four2a7701f.png" width="700">

**Fig.2 : MNIST dataset after dimension reduction to 2 as given in the previous notebook:**
<img src="https://s3-ap-southeast-1.amazonaws.com/he-public-data/mnist_plot53adb39.png" width="400">

## Challenge Question   
Use the VQC method from Aqua to classify the digits `4` and `9` as given in the dataset **challenge_dataset_4_9_dim3_training.csv** provided to you. 

## Rules and Guidelines

* Your `QuantumCircuit` can have a **maximum of 6 qubits**.
* **Cost of the circuit should be less than 2000**.  
* You should not change names of the functions `feature_map()` , `variational_circuit()`  and `return_optimal_params()`.
* All the functions must return the value types as mentioned. 
* All circuits must be Qiskit generated.
* Best of all submissions is considered for grading.

## Judging criteria 

* Primary judgement is based on the **accuracy of the model**, higher the better. **Accuracies which differ by less than 0.005 will be considered to be equal**. ex: Accuracies 0.7783 and 0.7741 will be considered to be equal.
* If the accuracies are tied, the tie will be broken using **cost of the circuit** as the metric, lower the better. 
* In the case that both accuracy of the model and cost of the circuit are equal, **time of submission** is taken into account, Earlier the better. 

_**Important Note:**_ The **leaderboard shown during the progress of the competition** will only display accuracy of the model and is **not the final leaderboard**. Breaking ties between accuracy of the model by considering lower **cost of circuit** will only be done after the competition ends. **The final leaderboard will be announced post the event** which will take into consideration cost of the circuit and time of submission. 

## Certificate Eligibility

Everyone who scores an **accuracy greater than 0.70 (i.e, 70%) will be eligible for a certificate**. 


An explanation on how to calculate the accuracy of the model and the cost of the circuit is given in the end inside the `grade()` function. Before you submit, make sure the grading function is running on your device. To save time you can also use the grading function provided to calculate the accuracy and circuit cost without having to submit your solution onto HackerEarth. Remember, your final score will be determined using the same grading methods as given in this notebook, but will be evaluated on unseen datapoints.

In [6]:
# installing a few dependencies
!pip install --upgrade seaborn==0.10.1
!pip install --upgrade scikit-learn==0.23.1
!pip install --upgrade matplotlib==3.2.0
!pip install --upgrade pandas==1.0.4
!pip install --upgrade qiskit==0.19.6 
!pip install --upgrade plotly==4.9.0

# the output will be cleared after installation
from IPython.display import clear_output
clear_output()

In [94]:
# we have imported a few libraries we thing might be useful 
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import Aer

from qiskit import *
import numpy as np
from qiskit.visualization import plot_bloch_multivector, plot_histogram
%matplotlib inline
import matplotlib.pyplot as plt

import time
from qiskit.circuit.library import ZZFeatureMap, ZFeatureMap, PauliFeatureMap, RealAmplitudes, EfficientSU2,ExcitationPreserving
from qiskit.aqua.utils import split_dataset_to_data_and_labels, map_label_to_class_name
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import VQC
from qiskit.aqua.components.optimizers import COBYLA


# The the write_and_run() magic function creates a file with the content inside the cell that it is run. 
# You have used this in previous exercises for creating your submission files. 
# It will be used for the same purpose here.

from IPython.core.magic import register_cell_magic
@register_cell_magic
def write_and_run(line, cell):
    argz = line.split()
    file = argz[-1]
    mode = 'w'
    with open(file, mode) as f:
        f.write(cell)
    get_ipython().run_cell(cell)

# Solution

## Data loading 

This notebook has helper functions and code snippets to save your time and help you concentrate on what's important: Increasing the accuracy of your model. Running the cell below will import the challenge dataset and will be available to you as `data`. Before running the cell below store the dataset in this file structure (or change the `data_path` accordingly):  

- `challenge_notebook.ipynb`
- `dataset`
    - `challenge_dataset_4_9.csv`


In [95]:
data_path='./dataset/'
data = np.loadtxt(data_path + "challenge_dataset_4_9_dim3_training.csv", delimiter=",")

# extracting the first column which contains the labels
data_labels = data[:, :1].reshape(data.shape[0],)
# extracting all the columns but the first which are our features
data_features = data[:, 1:]

In [96]:
data_labels


array([4., 4., 4., ..., 9., 9., 9.])

## Visualizing the dataset

Before we dive into solving the question it is always beneficial to look at the dataset pictographically. This will help us understand patterns which we could leverage when designing our feature maps and variational circuits for example.

In [97]:
import plotly.express as px
import pandas as pd

# creating a dataframe using pandas only for the purpose fo plotting
df = pd.DataFrame({'Component 0':data_features[:,0], 'Component 1':data_features[:,1], 
                   'Component 2':data_features[:,2], 'label':data_labels})

fig = px.scatter_3d(df, x='Component 0', y='Component 1', z='Component 2', color='label')
#fig.show()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Component 0  6000 non-null   float64
 1   Component 1  6000 non-null   float64
 2   Component 2  6000 non-null   float64
 3   label        6000 non-null   float64
dtypes: float64(4)
memory usage: 187.6 KB


## Extracting the training dataset

The given dataset has already been reduced in dimension and normalized, so, further pre-processing isn't techincally required. You can do so if you want to, but the testing dataset will be of the same dimension and normalisation as the training dataset provided. Training a dataset of size 6,000 will take multiple hours so you'll need to extract a subset of the dataset to use as a training dataset. The accuracy of the model may vary based on the datapoints and size of the training dataset you choose. Thus, experimenting with various sizes and datapoints will be necessary. For example, Increasing the training dataset size may increase the accuracy of the model however it will increase the training time as well.

Use the space below to extract your training dataset from `data`. For your convenience `data` has been segregated into `data_labels` and `data_features`.

* `data_labels` : 6,000 $\times$ 1 column vector with each entry either `4` or `9` 
* `data_features` : 6,000 $\times$ 3 matrix with each row having the feature corresponding to the label in `data_labels`

**Note:** This process was done in the previous [VQC notebook](https://github.com/Qiskit-Challenge-India/2020/blob/master/Day%206%2C%207%2C8/VQC_notebook.ipynb) with `0` and `1` labels and can be modified and used here as well. 

In [99]:

### WRITE YOUR CODE BETWEEN THESE LINES - START

four_datapoints = []
nine_datapoints = []
for i in range(6000):
    if data_labels[i] == 4:
         four_datapoints.append(data_features[i])

for i in range(6000):
    if data_labels[i] == 9:
         nine_datapoints.append(data_features[i]) 
four_datapoints = np.array(four_datapoints)
nine_datapoints = np.array(nine_datapoints)            
# do your classical pre-processing here
train_size = 500
test_size = 50
dp_size_four = 5
dp_size_nine = 5

four_train = four_datapoints[:train_size]
nine_train = nine_datapoints[:train_size]
four_test = four_datapoints[train_size + 1:train_size + test_size + 1]
nine_test = nine_datapoints[train_size + 1:train_size + test_size + 1]

 

# store your training and testing datasets to be input in the VQC optimizer in the "training_input" and 
# "testing_input" variables respectively. These variables will eb accessed whiile creating a VQC instance later. 
training_input = {'A':four_train, 'B':nine_train}
test_input = {'A':four_test, 'B':nine_test}


datapoints = []
dp_four = four_datapoints[train_size + test_size + 2:train_size + test_size + 2 + dp_size_four]
dp_nine = nine_datapoints[train_size + test_size + 2:train_size + test_size + 2 + dp_size_nine]
datapoints.append(np.concatenate((dp_four, dp_nine)))
dp_y = np.array([4, 4, 4, 4, 4, 9, 9, 9, 9, 9])
datapoints.append(dp_y)

class_to_label = {'A': 4, 'B': 9}
### WRITE YOUR CODE BETWEEN THESE LINES - END

## Building a Quantum Feature Map

Given below is the `feature_map()` function. It takes no input and has to return a feature map which is either a `FeatureMap` or `QuantumCircuit` object. In the previous notebook you've learnt how feature maps work and the process of using existing feature maps in Qiskit or creating your own. In the space given **inside the function** you have to create a feature map and return it.   


**IMPORTANT:** 
* If you require Qiskit import statements other than the ones provided in the cell below, please include them inside the appropriate space provided. **All additional import statements must be Qiskit imports.** 
* the first line of the cell below must be `%%write_and_run feature_map.py`. This function stores the content of the cell below in the file `feature_map.py`

In [100]:
%%write_and_run feature_map.py
# the write_and_run function writes the content in this cell into the file "feature_map.py"

### WRITE YOUR CODE BETWEEN THESE LINES - START

# import libraries that are used in the function below.
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import ZZFeatureMap, ZFeatureMap, PauliFeatureMap
    
### WRITE YOUR CODE BETWEEN THESE LINES - END

def feature_map(): 
    # BUILD FEATURE MAP HERE - START
    
    # import required qiskit libraries if additional libraries are required
    seed = 10598
    feature_dim = 3



  
    # build the feature map
    feature_map = PauliFeatureMap(feature_dimension=feature_dim, reps=1 ,entanglement = 'full', 
                                 paulis = ['Z', 'Y', 'ZZ']) # entanglement='linear'
    
    
    # BUILD FEATURE MAP HERE - END
    
    #return the feature map which is either a FeatureMap or QuantumCircuit object
    return feature_map

## Building a Variational Circuit

Given below is the `variational_circuit()` function. It takes no input and has to return a variational circuit which is either a `VariationalForm` or `QuantumCircuit` object. In the previous notebook you've learnt how variational circuits work and the process of using existing variational circuit in Qiskit or creating your own. You have to create a variational circuit in the space given **inside the function** and return it. You can find various variational circuits in the [Qiskit Circuit Library](https://qiskit.org/documentation/apidoc/circuit_library.html) under N-local circuits.

**IMPORTANT:** 
* If you require Qiskit import statements other than the ones provided in the cell below, please include them inside the appropriate space provided. **All additional import statements must be Qiskit imports.** 
* the first line of the cell below must be `%%write_and_run feature_map.py`. This function stores the content of the cell below in the file `variational_circuit.py`

In [101]:
%%write_and_run variational_circuit.py
# the write_and_run function writes the content in this cell into the file "variational_circuit.py"

### WRITE YOUR CODE BETWEEN THESE LINES - START
    
# import libraries that are used in the function below.
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import  RealAmplitudes, EfficientSU2 ,ExcitationPreserving
    
### WRITE YOUR CODE BETWEEN THESE LINES - END

def variational_circuit():
   
    # import required qiskit libraries if additional libraries are required
    feature_dim = 3
    # build the variational circuit
    var_circuit = EfficientSU2(feature_dim, reps=5, entanglement = 'full')

    #(feature_dim, reps=2, entanglement = 'full')
    

    # BUILD VARIATIONAL CIRCUIT HERE - END
    
    # return the variational circuit which is either a VaritionalForm or QuantumCircuit object
    return var_circuit

## Choosing a Classical Optimizer

In the `classical_optimizer()` function given below you will have to import the optimizer of your choice from [`qiskit.aqua.optimizers`](https://qiskit.org/documentation/apidoc/qiskit.aqua.components.optimizers.html) and return it. This function will not be called by the grading function `grade()` and thus the name of the function `classical_optimizer()`can be changed if needed. 

In [102]:
def classical_optimizer():
    # CHOOSE AND RETURN CLASSICAL OPTIMIZER OBJECT - START
    from qiskit.aqua.components.optimizers import COBYLA,GSLS
    # import the required clasical optimizer from qiskit.aqua.optimizers
    
    # create an optimizer object
    cls_opt = COBYLA(maxiter=500, tol=0.001)
    
    # CHOOSE AND RETURN CLASSICAL OPTIMIZER OBJECT - END
    return cls_opt

### Callback Function

The `VQC` class can take in a callback function to which the following parameters will be passed after every optimization cycle of the algorithm:

* `eval_count` : the evaulation counter
* `var_params` : value of parameters of the variational circuit
* `eval_val`  : current cross entropy cost 
* `index` : the batch index

In [103]:
def call_back_vqc(eval_count, var_params, eval_val, index):
    print("eval_count: {}".format(eval_count))
    print("var_params: {}".format(var_params))
    print("eval_val: {}".format(eval_val))
    print("index: {}".format(index))

## Optimization Step

This is where the whole VQC algorithm will come together. First we create an instance of the `VQC` class. 

In [104]:
# a fixed seed so that we get the same answer when the same input is given. 
seed = 10598
four_train = four_datapoints[:train_size]
nine_train = nine_datapoints[:train_size]
# setting our backend to qasm_simulator with the "statevector" method on. This particular setup is given as it was 
# found to perform better than most. Feel free to play around with different backend options.
backend = Aer.get_backend('qasm_simulator')
backend_options = {"method": "statevector"}

# creating a quantum instance using the backend and backend options taken before
quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=seed, seed_transpiler=seed, 
                                   backend_options=backend_options)

# creating a VQC instance which you will be used for training. Make sure you input the correct training_dataset and 
# testing_dataset as defined in your program.
feature_dim = 3
vqc = VQC(optimizer=classical_optimizer(), 
          feature_map=feature_map(), 
          var_form=variational_circuit(), 
          callback=call_back_vqc, 
          training_dataset=training_input,     # training_input must be initialized with your training dataset
          test_dataset=test_input)             # testing_input must be initialized with your testing dataset

Now, let's run the VQC classification routine

In [ ]:
start = time.process_time()

result = vqc.run(quantum_instance)

print("time taken: ")
print(time.process_time() - start)

print("testing success ratio: {}".format(result['testing_accuracy']))

eval_count: 0
var_params: [-0.21011912  0.14511795  0.87562948 -0.36215402  0.96290155  0.98512343
 -1.43370391 -1.06964658 -0.96985244 -0.57978391  0.13938416  1.15179235
  1.0624462  -0.41926938 -0.08816506  1.29447967 -1.29703284 -0.72662756
  1.275422   -0.97022886 -1.22429247 -1.76355547  0.35637199  0.2305944
  1.10996634  0.61483419  1.0432403  -0.36454176  1.89708569 -0.60419621
  0.70667302  0.51129727  1.24546024 -0.23319966  1.44738841 -1.01541443]
eval_val: 0.8833024027922237
index: 0
eval_count: 1
var_params: [ 0.78988088  0.14511795  0.87562948 -0.36215402  0.96290155  0.98512343
 -1.43370391 -1.06964658 -0.96985244 -0.57978391  0.13938416  1.15179235
  1.0624462  -0.41926938 -0.08816506  1.29447967 -1.29703284 -0.72662756
  1.275422   -0.97022886 -1.22429247 -1.76355547  0.35637199  0.2305944
  1.10996634  0.61483419  1.0432403  -0.36454176  1.89708569 -0.60419621
  0.70667302  0.51129727  1.24546024 -0.23319966  1.44738841 -1.01541443]
eval_val: 0.7574973295976608
index

eval_count: 17
var_params: [ 0.78988088  1.14511795  1.87562948 -0.36215402  1.96290155  0.98512343
 -1.43370391 -0.06964658 -0.96985244 -0.57978391  0.13938416  1.15179235
  1.0624462  -0.41926938 -0.08816506  1.29447967 -0.29703284 -0.72662756
  1.275422   -0.97022886 -1.22429247 -1.76355547  0.35637199  0.2305944
  1.10996634  0.61483419  1.0432403  -0.36454176  1.89708569 -0.60419621
  0.70667302  0.51129727  1.24546024 -0.23319966  1.44738841 -1.01541443]
eval_val: 0.698291379506964
index: 17
eval_count: 18
var_params: [ 0.78988088  1.14511795  1.87562948 -0.36215402  1.96290155  0.98512343
 -1.43370391 -0.06964658 -0.96985244 -0.57978391  0.13938416  1.15179235
  1.0624462  -0.41926938 -0.08816506  1.29447967 -1.29703284  0.27337244
  1.275422   -0.97022886 -1.22429247 -1.76355547  0.35637199  0.2305944
  1.10996634  0.61483419  1.0432403  -0.36454176  1.89708569 -0.60419621
  0.70667302  0.51129727  1.24546024 -0.23319966  1.44738841 -1.01541443]
eval_val: 0.6928818904519671
ind

eval_count: 34
var_params: [ 0.78988088  1.14511795  1.87562948 -0.36215402  1.96290155  0.98512343
 -1.43370391 -0.06964658 -0.96985244 -0.57978391  0.13938416  1.15179235
  1.0624462  -0.41926938 -0.08816506  1.29447967 -1.29703284 -0.72662756
  1.275422    0.02977114 -1.22429247 -1.76355547  0.35637199  0.2305944
  1.10996634  0.61483419  1.0432403  -0.36454176  1.89708569 -0.60419621
  0.70667302  0.51129727  1.24546024  0.76680034  1.44738841 -1.01541443]
eval_val: 0.6586774452175702
index: 34
eval_count: 35
var_params: [ 0.78988088  1.14511795  1.87562948 -0.36215402  1.96290155  0.98512343
 -1.43370391 -0.06964658 -0.96985244 -0.57978391  0.13938416  1.15179235
  1.0624462  -0.41926938 -0.08816506  1.29447967 -1.29703284 -0.72662756
  1.275422    0.02977114 -1.22429247 -1.76355547  0.35637199  0.2305944
  1.10996634  0.61483419  1.0432403  -0.36454176  1.89708569 -0.60419621
  0.70667302  0.51129727  1.24546024 -0.23319966  2.44738841 -1.01541443]
eval_val: 0.7342610214167662
in

eval_count: 49
var_params: [ 1.25535468  1.34485431  2.44177908 -0.43328318  2.53338519  1.43085833
 -0.63315071  0.04216758 -1.32195654 -0.75137761  0.00802035  0.88760843
  1.01310768  0.06485267 -0.55616387  1.03125837 -0.8358992  -0.98686675
  1.03633973  0.04909834 -1.39081134 -1.80271151  0.1585123   0.06136636
  0.96903256  0.22675153  1.0432403  -0.36454176  1.89708569 -0.70205861
  0.51767648  0.43936205  0.89898002 -0.33738141  0.95564889 -1.32025187]
eval_val: 0.6185623772012265
index: 49
eval_count: 50
var_params: [ 1.20895421  1.35438013  2.42230748 -0.40591234  2.48634401  1.39484784
 -0.64588564  0.0102067  -1.23883193 -0.70232938  0.0455693   1.3140066
  1.02721057 -0.60919293 -0.51041271  1.03265211 -1.50269438 -1.26868291
  1.10467885  0.04357386 -1.3432137  -1.79151918  0.21506838  0.10973839
  1.00931698  0.33768082  1.0432403  -0.36454176  1.89708569 -0.67408568
  0.57169913  0.45992397  0.9980177  -0.30760217  1.09620738 -1.23311734]
eval_val: 0.5949673744295065
i

eval_count: 66
var_params: [ 1.2023324   1.33923448  2.42523598 -0.40430634  2.50139709  1.40993889
 -0.63626339  0.00544691 -1.25431067 -0.68904966  0.04506871  0.9500283
  1.00508968 -0.59689411 -0.50413549  1.04503901 -1.51116309 -0.92115022
  1.08916941  0.09974867 -0.97187214 -1.84965079 -0.00946841 -0.07588751
  1.30016871  0.32749585  0.98897278 -0.41880927  1.84281817 -0.7380107
  0.74373532  0.39855756  0.9926851  -0.37215081  1.10198746 -1.11505868]
eval_val: 0.5862945878553619
index: 66
eval_count: 67
var_params: [ 1.21385095  1.35694449  2.42312802 -0.40830339  2.48660086  1.39427284
 -0.6470158   0.01105548 -1.24169099 -0.70363195  0.04457211  0.96111716
  1.02683604 -0.61121169 -0.51518645  1.02986907 -1.50488041 -0.91445563
  1.10286397  0.03183717 -1.33343519 -1.77978248 -0.00568552 -0.07347355
  1.01912474  0.33473487  1.05497699 -0.35280506  1.90882238 -0.84715011
  0.74064831  0.47166066  0.99538756 -0.12740762  1.09247458 -1.11871843]
eval_val: 0.5917653066670703
in

...

eval_count: 313
var_params: [ 1.20209241  1.43155908  2.46421611 -0.20926391  2.54683504  1.36374277
 -0.80182206 -0.14362549 -1.12535841 -0.54773102  0.25005492  1.05369202
  1.07238772 -0.61225044 -0.61448451  1.02644578 -1.64008369 -0.98459422
  1.28772168 -0.2077375  -1.09886221 -1.70441894  0.01114277 -0.12442449
  0.91544713  0.77350981  1.17521026 -0.44264798  1.82551509 -0.5092688
  0.92240562  0.53065405  1.08247317 -0.08820339  1.26791156 -0.91569886]
eval_val: 0.5653767009840411
index: 313
eval_count: 314
var_params: [ 1.20396899  1.42993438  2.47458814 -0.20765326  2.55360637  1.37159979
 -0.79725805 -0.14670093 -1.12414408 -0.54225586  0.25735743  1.05142395
  1.08091392 -0.60955096 -0.5988048   1.02468583 -1.63737821 -0.98922708
  1.2821216  -0.21328424 -1.09416171 -1.69479417  0.0095723  -0.12888391
  0.92046212  0.7679414   1.18242741 -0.43754135  1.82198069 -0.50701715
  0.92378057  0.5323358   1.07083151 -0.09174303  1.26349364 -0.91340272]
eval_val: 0.565498392165612

eval_count: 327
var_params: [ 1.21568031  1.41513502  2.47014883 -0.18780268  2.54252406  1.35491142
 -0.79267413 -0.15545367 -1.0984124  -0.52127231  0.24339365  1.08204881
  1.09336852 -0.59044786 -0.6108766   1.01899128 -1.62468455 -0.9762425
  1.28111162 -0.208108   -1.1097789  -1.71021595  0.00895058 -0.12660431
  0.90651416  0.79003448  1.16374865 -0.45190946  1.83306049 -0.51169321
  0.9381771   0.52322037  1.0787557  -0.06773682  1.26656153 -0.91884162]
eval_val: 0.5648588801760764
index: 327
eval_count: 328
var_params: [ 1.21612079  1.43025973  2.46707197 -0.18560173  2.53793931  1.35114503
 -0.78866059 -0.15012996 -1.08618078 -0.52412003  0.24912475  1.09029008
  1.08948075 -0.59575544 -0.60926904  1.01718984 -1.62442249 -0.97517944
  1.28322317 -0.21184875 -1.10246631 -1.71538467  0.00679794 -0.13621003
  0.9124075   0.79993103  1.17227961 -0.45456512  1.82813342 -0.50484325
  0.93891557  0.52527089  1.07996913 -0.07767269  1.27203311 -0.91963355]
eval_val: 0.564582925311038

eval_count: 343
var_params: [ 1.2036468   1.4368524   2.46650195 -0.18289775  2.53666649  1.35954451
 -0.78824947 -0.15321661 -1.07746869 -0.51392614  0.26546273  1.0933708
  1.09963907 -0.60660477 -0.61873411  1.01186379 -1.61013692 -0.96976426
  1.26440575 -0.21088359 -1.1334157  -1.7067101   0.00737429 -0.13060389
  0.92374146  0.81939396  1.17727493 -0.453947    1.82673534 -0.49068178
  0.93434193  0.5487349   1.10097738 -0.07193484  1.29052576 -0.94840594]
eval_val: 0.56388883625594
index: 343
eval_count: 344
var_params: [ 1.20739706  1.42631728  2.46634367 -0.18878908  2.53859415  1.34528168
 -0.79468764 -0.15167159 -1.07055166 -0.51114618  0.26165065  1.08626694
  1.09605393 -0.6026695  -0.6172689   1.0233141  -1.60486122 -0.97361659
  1.26562914 -0.20689318 -1.12457709 -1.70731741  0.00482623 -0.13676702
  0.91949866  0.81086231  1.17957057 -0.44962523  1.82064917 -0.48972403
  0.93234631  0.54711698  1.08950106 -0.07779092  1.28626561 -0.94728162]
eval_val: 0.5640578601682866


eval_count: 359
var_params: [ 1.18001262  1.42653396  2.46188405 -0.18462156  2.52219413  1.33361648
 -0.80141525 -0.14291665 -1.07361425 -0.52708772  0.26058981  1.09051675
  1.08910312 -0.59546975 -0.64071641  1.01985955 -1.61851816 -0.98674866
  1.24362659 -0.18368701 -1.11693718 -1.7035319   0.01056137 -0.13490768
  0.93103533  0.82272612  1.1778672  -0.46321622  1.8072682  -0.50358333
  0.9372398   0.53687312  1.11216692 -0.07842967  1.29197035 -0.94588178]
eval_val: 0.5634043044142006
index: 359
eval_count: 360
var_params: [ 1.18039707  1.4220135   2.46470391 -0.18001894  2.5323237   1.331309
 -0.80125294 -0.15014881 -1.06123934 -0.52378404  0.26153213  1.08882328
  1.09374179 -0.60593361 -0.63495426  1.01975141 -1.62256307 -0.97790038
  1.25400286 -0.19181513 -1.11270055 -1.70181454  0.00993768 -0.13212952
  0.93038138  0.81701904  1.17666189 -0.46040473  1.81694817 -0.50400121
  0.93118076  0.52848477  1.10832869 -0.0712448   1.29998569 -0.94860009]
eval_val: 0.5631130615735309

eval_count: 375
var_params: [ 1.18438318e+00  1.43430916e+00  2.45699872e+00 -1.88461497e-01
  2.52032811e+00  1.32293687e+00 -8.11755270e-01 -1.39553890e-01
 -1.04428926e+00 -5.16079363e-01  2.68093429e-01  1.10404365e+00
  1.09288642e+00 -5.86686712e-01 -6.48097671e-01  1.00393459e+00
 -1.63861294e+00 -9.76084631e-01  1.26846615e+00 -2.05762259e-01
 -1.11624600e+00 -1.68766596e+00 -1.42366312e-03 -1.38730048e-01
  9.16873951e-01  8.05611773e-01  1.18305372e+00 -4.57581605e-01
  1.81695329e+00 -5.08506491e-01  9.26042444e-01  5.16633210e-01
  1.12600692e+00 -7.80878374e-02  1.31480746e+00 -9.64652489e-01]
eval_val: 0.5631533341039551
index: 375
eval_count: 376
var_params: [ 1.18407447  1.43017943  2.46421066 -0.18401969  2.52308283  1.32405687
 -0.81158386 -0.14122379 -1.05147542 -0.51950891  0.27151901  1.1102088
  1.08103989 -0.58828766 -0.64243038  1.01405425 -1.63586478 -0.97784448
  1.26523342 -0.20426832 -1.11807483 -1.68595947 -0.00258716 -0.13358902
  0.92683115  0.80509532  1

## Storing the optimal parameters for grading

Once the training step of the vqc algorithm is done we obtain the optimal parameters for our specific variational form. For the grading function to be able to access these optimal parameters you will need to follow the steps below. 

* **Step 1**: Run the cell below with `print(repr(vqc.optimal_params))`. 
* **Step 2**: Copy the matrix of optimal parameters and store it in the variable `optimal_parameters` inside the function `return_optimal_params()` in the next cell. This will enable us to extract it while calculating the accuracy your the model during grading. Given below is a pictographical explanation of the same:  

<img src="https://s3-ap-southeast-1.amazonaws.com/he-public-data/opt_params456b075.png" width="800">

In [87]:
print(repr(vqc.optimal_params))

array([ 1.24737937, -1.48250781,  0.69173882,  2.00958784,  0.3636373 ,
       -0.58930166, -0.14721266, -0.57709828, -1.65739815,  1.37887435,
        0.24428202, -0.84546396, -0.64284346, -2.03318745, -3.7424252 ,
        1.01973101,  1.16275005,  0.86952223, -1.2584524 , -1.04100862,
        0.98028603, -0.38108364, -0.77729507, -1.12688145,  0.98596223,
       -2.33247688,  0.46020213, -0.13777206,  0.2579294 ,  1.18174236,
       -0.84713412,  0.01589791,  1.73991139,  0.04785303,  2.14051929,
       -2.01119941])


In [88]:
%%write_and_run optimal_params.py
# # the write_and_run function writes the content in this cell into the file "optimal_params.py"

### WRITE YOUR CODE BETWEEN THESE LINES - START
    
# import libraries that are used in the function below.
import numpy as np
    
### WRITE YOUR CODE BETWEEN THESE LINES - END

def return_optimal_params():
    # STORE THE OPTIMAL PARAMETERS AS AN ARRAY IN THE VARIABLE optimal_parameters 
    
    optimal_parameters =[ 1.24737937, -1.48250781,  0.69173882,  2.00958784,  0.3636373 ,
       -0.58930166, -0.14721266, -0.57709828, -1.65739815,  1.37887435,
        0.24428202, -0.84546396, -0.64284346, -2.03318745, -3.7424252 ,
        1.01973101,  1.16275005,  0.86952223, -1.2584524 , -1.04100862,
        0.98028603, -0.38108364, -0.77729507, -1.12688145,  0.98596223,
       -2.33247688,  0.46020213, -0.13777206,  0.2579294 ,  1.18174236,
       -0.84713412,  0.01589791,  1.73991139,  0.04785303,  2.14051929,
       -2.01119941]
    # STORE THE OPTIMAL PARAMETERS AS AN ARRAY IN THE VARIABLE optimal_parameters 
    return np.array(optimal_parameters)

## Submission

Before we go any further, check that you have the three files `feature_map.py`, `variational_circuit.py` and `optimal_params.py` in the **same working directory as this notebook**. If you do not, then go back to the start and run the notebook making sure you have filled in the code where its required. When you run the cell below, all the three files `feature_map.py`, `variational_circuit.py` and `optimal_params.py` are combined into one file named **"answer.py"**. Now your working directory will have four python (.py) files out of which **"answer.py"** is the submission file: 
* `answer.py` <- upload this file onto HackerEarth and click on "Submit and Evaluate"
* `feature_map.py`
* `variational_circuit.py`
* `optimal_params.py`

In [89]:
solution = ['feature_map.py','variational_circuit.py','optimal_params.py']
file = open("answer.py","w")
file.truncate(0)
for i in solution:    
    with open(i) as f:
        with open("answer.py", "a") as f1:
            for line in f:
                f1.write(line)
file.close()

## Grading Function

Given below is the grading function that we shall use to grade your submission with a test dataset that is of the same format as `challenge_dataset_4_9.csv`. You can use it to grade your submission by extracting a few points out of the `challenge_dataset_4_9.csv` to get a basic idea of how your model is performing. 

In [90]:
#imports required for the grading function 
from qiskit import *
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import VQC
from qiskit.aqua.components.feature_maps import FeatureMap
from qiskit.aqua.components.variational_forms import VariationalForm
import numpy as np

### Working of the grading function

The grading function `grade()` takes as **input**: 

* `test_data`: (`np.ndarray`) -- **no. of datapoints $\times$ dimension of data** : the datapoints against which we want to test our model. 


* `test_labels`: (`np.ndarray`) -- **no. of datapoints $\times$ 1** : A column vector with each entry either 0 or 1 as entries.


* `feature_map`: (`QuantumCircuit` or `FeatureMap`) -- A quantum feature map which is the output of `feature_map()` defined earlier.


* `variational_form`: (`QuantumCircuit` or `VariationalForm`) -- A variational form which is the output of `variational_circuit()` defined earlier.


* `optimal_params`: (`numpy.ndarray`) -- the optimal parameters obtained after running the VQC algorithm above. These are the values obtained when the function `return_optimal_params()` is run. 


* `find_circuit_cost` : (`bool`) -- Calculates the circuit cost if set to `True`. Circuit cost is calculated by converting the circuit to the basis gate set `\[ 'u3', 'cx'\]` and then applying the formula **cost = 1$\times$(no.of u3 gates) + 10$\times$(no.of cx gates)**.


* `verbose` : (`bool`) -- prints the result message if set to `True`.

And gives as **output**: 

* `model_accuracy` : (`numpy.float64`) -- percent accuracy of the model. 


* `circuit_cost`: (`int`) -- circuit cost as explained above.


* `ans`: (`tuple`) -- Output of the `VQC.predict()` method. 


* `result_msg`: (`str`) -- Result message which also outputs the error message in case of one.


* `unrolled_circuit`: (`QuantumCircuit` or `None`) -- the circuit obtained after unrolling the full VQC circuit and substituting the optimal parameters to the basis gate set `\[ 'u3', 'cx'\]`.

**Note:** if you look inside the `grade()` function in Section 2 you'll see that we have initialized a COBYLA optimizer though the prediction step will not require one. Similarily we have given a dataset to `training dataset`. Both of these are dummy variables. The reason for this is because these are not optional variables the `VQC` class instantiation.  

In [91]:
def grade(test_data, test_labels, feature_map, variational_form, optimal_params, find_circuit_cost=True, verbose=True):
    seed = 10598
    model_accuracy = None 
    circuit_cost=None 
    ans = None
    unrolled_circuit = None
    result_msg=''
    data_dim = np.array(test_data).shape[1]
    dataset_size = np.array(test_data).shape[0]
    dummy_training_dataset=training_input = {'A':np.ones((2,data_dim)), 'B':np.ones((2, data_dim))}
    
    # converting 4's to 0's and 9's to 1's for checking 
    test_labels_transformed = np.where(test_labels==4, 0., 1.)
    max_qubit_count = 6
    max_circuit_cost = 2000
    
    # Section 1
    if feature_map is None:
        result_msg += 'feature_map variable is None. Please submit a valid entry' if verbose else ''
    elif variational_form is None: 
        result_msg += 'variational_form variable is None. Please submit a valid entry' if verbose else ''
    elif optimal_params is None: 
        result_msg += 'optimal_params variable is None. Please submit a valid entry' if verbose else ''
    elif test_data is None: 
        result_msg += 'test_data variable is None. Please submit a valid entry' if verbose else ''
    elif test_labels is None: 
        result_msg += 'test_labels variable is None. Please submit a valid entry' if verbose else ''
    elif not isinstance(feature_map, (QuantumCircuit, FeatureMap)):
        result_msg += 'feature_map variable should be a QuantumCircuit or a FeatureMap not (%s)' % \
                      type(feature_map) if verbose else ''
    elif not isinstance(variational_form, (QuantumCircuit, VariationalForm)):
        result_msg += 'variational_form variable should be a QuantumCircuit or a VariationalForm not (%s)' % \
                      type(variational_form) if verbose else ''
    elif not isinstance(test_data, np.ndarray):
        result_msg += 'test_data variable should be a numpy.ndarray not (%s)' % \
                      type(test_data) if verbose else ''
    elif not isinstance(test_labels, np.ndarray):
        result_msg += 'test_labels variable should be a numpy.ndarray not (%s)' % \
                      type(test_labels) if verbose else ''
    elif not isinstance(optimal_params, np.ndarray):
        result_msg += 'optimal_params variable should be a numpy.ndarray not (%s)' % \
                      type(optimal_params) if verbose else ''
    elif not dataset_size == test_labels_transformed.shape[0]:
        result_msg += 'Dataset size and label array size must be equal'
    # Section 2
    else:
        
        # setting up COBYLA optimizer as a dummy optimizer
        from qiskit.aqua.components.optimizers import COBYLA
        dummy_optimizer = COBYLA()

        # setting up the backend and creating a quantum instance
        backend = Aer.get_backend('qasm_simulator')
        backend_options = {"method": "statevector"}
        quantum_instance = QuantumInstance(backend, 
                                           shots=2000, 
                                           seed_simulator=seed, 
                                           seed_transpiler=seed, 
                                           backend_options=backend_options)

        # creating a VQC instance and running the VQC.predict method to get the accuracy of the model 
        vqc = VQC(optimizer=dummy_optimizer, 
                  feature_map=feature_map, 
                  var_form=variational_form, 
                  training_dataset=dummy_training_dataset)
        
        from qiskit.transpiler import PassManager
        from qiskit.transpiler.passes import Unroller
        pass_ = Unroller(['u3', 'cx'])
        pm = PassManager(pass_)
        # construct circuit with first datapoint
        circuit = vqc.construct_circuit(data[0], optimal_params)
        unrolled_circuit = pm.run(circuit)
        gates = unrolled_circuit.count_ops()
        if 'u3' in gates: 
            circuit_cost = gates['u3']
        if 'cx' in gates: 
            circuit_cost+= 10*gates['cx']
        
        if circuit.num_qubits > max_qubit_count:
            result_msg += 'Your quantum circuit is using more than 6 qubits. Reduce the number of qubits used and try again.'
        elif circuit_cost > max_circuit_cost:
            result_msg += 'The cost of your circuit is exceeding the maximum accpetable cost of 2000. Reduce the circuit cost and try again.'
        else: 
            
            ans = vqc.predict(test_data, quantum_instance=quantum_instance, params=np.array(optimal_params))
            model_accuracy = np.sum(np.equal(test_labels_transformed, ans[1]))/len(ans[1])

            result_msg += 'Accuracy of the model is {}'.format(model_accuracy) if verbose else ''
            result_msg += ' and circuit cost is {}'.format(circuit_cost) if verbose else ''
            
    return model_accuracy, circuit_cost, ans, result_msg, unrolled_circuit

## Process of grading using a dummy grading dataset

Let us create a dummy grading dataset with features and labels `grading_features` and `grading_labels` created from the last 2000 datapoints from `data_features` and `data_labels`so that we can a rough estimate of our accuaracy. It must be noted that this may not be a balanced dataset, i.e, may not have equal number of `4`'s and `9`'s and is not best practice. This is only given for the purpose of the demo of `grade()` function. In the final scoring done on HackerEarth, the testing dataset used will have a balanced number of class labels `4` and `9`.

In [92]:
grading_dataset_size=2000    # this value is not per digit but in total
grading_features = data_features[-grading_dataset_size:]
grading_labels = data_labels[-grading_dataset_size:]

In [93]:
start = time.process_time()

accuracy, circuit_cost, ans, result_msg, full_circuit  =  grade(test_data=grading_features, 
                                                                test_labels=grading_labels, 
                                                                feature_map=feature_map(), 
                                                                variational_form=variational_circuit(), 
                                                                optimal_params=return_optimal_params())

print("time taken: {} seconds".format(time.process_time() - start))
print(result_msg)

time taken: 43.828125 seconds
Accuracy of the model is 0.74 and circuit cost is 330


You can also check your **accuracy**, **circuit_cost** and **full_circuit** which is the result of combining the feature map and variational circuit and unrolling into the basis \['u3', 'cx'\].

In [37]:
print("Accuracy of the model: {}".format(accuracy))
print("Circuit Cost: {}".format(circuit_cost))
print("The complete unrolled circuit: ")
full_circuit.draw()

Accuracy of the model: 0.0
Circuit Cost: 540
The complete unrolled circuit: 


┌───────────────┐  ┌───────────┐   ┌─────────────────────┐»
q_0: ┤ U3(pi/2,0,pi) ├──┤ U3(0,0,8) ├───┤ U3(pi/2,-pi/2,pi/2) ├»
     ├───────────────┤┌─┴───────────┴──┐├─────────────────────┤»
q_1: ┤ U3(pi/2,0,pi) ├┤ U3(0,0,1.6456) ├┤ U3(pi/2,-pi/2,pi/2) ├»
     ├───────────────┤├────────────────┤├─────────────────────┤»
q_2: ┤ U3(pi/2,0,pi) ├┤ U3(0,0,1.8033) ├┤ U3(pi/2,-pi/2,pi/2) ├»
     └───────────────┘└────────────────┘└─────────────────────┘»
c_0: ══════════════════════════════════════════════════════════»
                                                               »
c_1: ══════════════════════════════════════════════════════════»
                                                               »
c_2: ══════════════════════════════════════════════════════════»
                                                               »
«       ┌───────────┐   ┌──────────────────────┐                             »
«q_0: ──┤ U3(0,0,8) ├───┤ U3(-pi/2,-pi/2,pi/2) ├──■───────────────────────■──»
«     ┌─┴───────────┴──┐├──────────────────────┤┌─┴─┐┌─────────────────┐┌─┴─┐»
«q_1: ┤ U3(0,0,1.6456) ├┤ U3(-pi/2,-pi/2,pi/2) ├┤ X ├┤ U3(0,0,-3.9809) ├┤ X ├»
«     ├────────────────┤├──────────────────────┤└───┘└─────────────────┘└───┘»
«q_2: ┤ U3(0,0,1.8033) ├┤ U3(-pi/2,-pi/2,pi/2) ├─────────────────────────────»
«     └────────────────┘└──────────────────────┘                             »
«c_0: ═══════════════════════════════════════════════════════════════════════»
«                                                                            »
«c_1: ═══════════════════════════════════════════════════════════════════════»
«                                                                            »
«c_2: ═══════════════════════════════════════════════════════════════════════»
«                                                                            »
«                                  ┌────────────────┐┌───────────────┐      »
«q_0: ──■───────────────────────■──┤ U3(0,0,1.4097) ├┤ U3(pi/2,0,pi) ├──────»
«       │                       │  └────────────────┘└───────────────┘      »
«q_1: ──┼───────────────────────┼──────────■─────────────────────────────■──»
«     ┌─┴─┐┌─────────────────┐┌─┴─┐      ┌─┴─┐       ┌────────────────┐┌─┴─┐»
«q_2: ┤ X ├┤ U3(0,0,-3.8456) ├┤ X ├──────┤ X ├───────┤ U3(0,0,10.388) ├┤ X ├»
«     └───┘└─────────────────┘└───┘      └───┘       └────────────────┘└───┘»
«c_0: ══════════════════════════════════════════════════════════════════════»
«                                                                           »
«c_1: ══════════════════════════════════════════════════════════════════════»
«                                                                           »
«c_2: ══════════════════════════════════════════════════════════════════════»
«                                                                           »
«                                                                        »
«q_0: ───────────────────────────────────────■────────────────────────■──»
«      ┌────────────────┐ ┌───────────────┐┌─┴─┐┌──────────────────┐┌─┴─┐»
«q_1: ─┤ U3(0,0,1.2838) ├─┤ U3(pi/2,0,pi) ├┤ X ├┤ U3(0,0,-0.51597) ├┤ X ├»
«     ┌┴────────────────┴┐├───────────────┤└───┘└──────────────────┘└───┘»
«q_2: ┤ U3(0,0,-0.33333) ├┤ U3(pi/2,0,pi) ├──────────────────────────────»
«     └──────────────────┘└───────────────┘                              »
«c_0: ═══════════════════════════════════════════════════════════════════»
«                                                                        »
«c_1: ═══════════════════════════════════════════════════════════════════»
«                                                                        »
«c_2: ═══════════════════════════════════════════════════════════════════»
«                                                                        »
«     ┌───────────────┐┌─────────────────────┐                              »
«q_0: ┤ U3(pi/2,0,pi) ├┤ U3(pi/2,-pi/2,pi/2) ├──■─────────────────